# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlite3
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sandy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sandy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [33]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
X = df['message']
Y = df.drop(['message','original','genre','id'], axis=1)

### 2. Write a tokenization function to process your text data

In [34]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
#NOT WITH PIPELINE

#X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

#vect = CountVectorizer(tokenizer=tokenize)
#tfidf = TfidfTransformer()
#clf = MultiOutputClassifier(RandomForestClassifier())

# train classifier
#X_train_counts = vect.fit_transform(X_train)
#X_train_tfidf = tfidf.fit_transform(X_train_counts)
#clf.fit(X_train_tfidf, Y_train)

# predict on test data
#X_test_counts = vect.transform(X_test)
#X_test_tfidf = tfidf.transform(X_test_counts)
#y_pred = clf.predict(X_test_tfidf)

In [35]:
# build pipeline
pipeline_rf_base = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
#split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train classifier
pipeline_rf_base.fit(X_train, y_train)

# predict on test data
y_pred = pipeline_rf_base.predict(X_test) 

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# report the f1 score, precision and recall for each output category of the dataset
y_test_df = y_test
y_pred_df = pd.DataFrame(data=y_pred, columns= Y.columns.values)

for col_name in Y.columns.values:
    print(classification_report(y_test_df[col_name].values, y_pred_df[col_name].values))

              precision    recall  f1-score   support

           0       0.73      0.29      0.41      1503
           1       0.81      0.97      0.88      5001
           2       0.67      0.16      0.26        50

    accuracy                           0.81      6554
   macro avg       0.74      0.47      0.52      6554
weighted avg       0.79      0.81      0.77      6554

              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5408
           1       0.88      0.41      0.56      1146

    accuracy                           0.89      6554
   macro avg       0.88      0.70      0.75      6554
weighted avg       0.89      0.89      0.87      6554

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0

c:\users\sandy\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6332
           1       0.56      0.05      0.08       222

    accuracy                           0.97      6554
   macro avg       0.76      0.52      0.53      6554
weighted avg       0.95      0.97      0.95      6554

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6554

    accuracy                           1.00      6554
   macro avg       1.00      1.00      1.00      6554
weighted avg       1.00      1.00      1.00      6554

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6169
           1       0.84      0.25      0.39       385

    accuracy                           0.95      6554
   macro avg       0.90      0.63      0.68      6554
weighted avg       0.95      0.95      0.94      6554

              precision    recall  f1-score   support

           0     


              precision    recall  f1-score   support

           0       0.86      0.99      0.92      5283
           1       0.86      0.33      0.47      1271

    accuracy                           0.86      6554
   macro avg       0.86      0.66      0.70      6554
weighted avg       0.86      0.86      0.83      6554



### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
# perform gridsearchcv for random forest classifier

#pipeline.get_params().keys()

#parameters = {
    #'vect__ngram_range': ((1,1),(1,2)),
    #'tfidf__use_idf': (True, False)
    #}

#cv = GridSearchCV(pipeline, param_grid=parameters)

#cv.fit(X_train,y_train)

#print("\nBest Parameters:", cv.best_params_) 

In [36]:
#build a new pipeline with best parameters - GridSearchCV (params = 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2))

pipeline_rf_gridsearchcv = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf='False')),
    ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [37]:
# train classifier
pipeline_rf_gridsearchcv.fit(X_train, y_train)

# predict on test data
y_pred = pipeline_rf_gridsearchcv.predict(X_test)

# report the f1 score, precision and recall for each output category of the dataset
y_test_df = y_test
y_pred_df = pd.DataFrame(data=y_pred, columns= Y.columns.values)

for col_name in Y.columns.values:
    print(classification_report(y_test_df[col_name].values, y_pred_df[col_name].values))

              precision    recall  f1-score   support

           0       0.71      0.29      0.41      1503
           1       0.81      0.96      0.88      5001
           2       0.46      0.12      0.19        50

    accuracy                           0.80      6554
   macro avg       0.66      0.46      0.49      6554
weighted avg       0.79      0.80      0.77      6554

              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5408
           1       0.88      0.45      0.59      1146

    accuracy                           0.89      6554
   macro avg       0.89      0.72      0.77      6554
weighted avg       0.89      0.89      0.88      6554

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0

c:\users\sandy\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.76      0.88      0.81      3854
           1       0.77      0.60      0.68      2700

    accuracy                           0.76      6554
   macro avg       0.77      0.74      0.74      6554
weighted avg       0.76      0.76      0.76      6554

              precision    recall  f1-score   support

           0       0.92      1.00      0.96      6016
           1       0.83      0.02      0.04       538

    accuracy                           0.92      6554
   macro avg       0.88      0.51      0.50      6554
weighted avg       0.91      0.92      0.88      6554

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6215
           1       0.65      0.04      0.08       339

    accuracy                           0.95      6554
   macro avg       0.80      0.52      0.53      6554
weighted avg       0.93      0.95      0.93      6554

              preci

              precision    recall  f1-score   support

           0       0.92      1.00      0.96      5940
           1       0.79      0.17      0.28       614

    accuracy                           0.92      6554
   macro avg       0.86      0.58      0.62      6554
weighted avg       0.91      0.92      0.89      6554

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6477
           1       0.00      0.00      0.00        77

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5929
           1       0.91      0.44      0.59       625

    accuracy                           0.94      6554
   macro avg       0.93      0.72      0.78      6554
weighted avg       0.94      0.94      0.93      6554

              preci

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
#build a new pipeline with KNN algorithms

pipeline_knn_base = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

In [12]:
# train classifier
pipeline_knn_base.fit(X_train, y_train)

# predict on test data
y_pred = pipeline_knn_base.predict(X_test)

# report the f1 score, precision and recall for each output category of the dataset
y_test_df = y_test
y_pred_df = pd.DataFrame(data=y_pred, columns= Y.columns.values)

for col_name in Y.columns.values:
    print(classification_report(y_test_df[col_name].values, y_pred_df[col_name].values))

              precision    recall  f1-score   support

           0       0.53      0.42      0.47      1503
           1       0.83      0.89      0.86      5001
           2       0.55      0.24      0.33        50

    accuracy                           0.78      6554
   macro avg       0.64      0.52      0.55      6554
weighted avg       0.76      0.78      0.77      6554

              precision    recall  f1-score   support

           0       0.88      0.98      0.92      5408
           1       0.78      0.35      0.48      1146

    accuracy                           0.87      6554
   macro avg       0.83      0.66      0.70      6554
weighted avg       0.86      0.87      0.85      6554

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0

c:\users\sandy\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6452
           1       0.45      0.05      0.09       102

    accuracy                           0.98      6554
   macro avg       0.72      0.52      0.54      6554
weighted avg       0.98      0.98      0.98      6554

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6394
           1       0.60      0.06      0.10       160

    accuracy                           0.98      6554
   macro avg       0.79      0.53      0.55      6554
weighted avg       0.97      0.98      0.97      6554

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6489
           1       1.00      0.02      0.03        65

    accuracy                           0.99      6554
   macro avg       1.00      0.51      0.51      6554
weighted avg       0.99      0.99      0.99      6554

              prec

### 9. Export your model as a pickle file

In [ ]:
#https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/

In [13]:
# rf_base

# save the model to disk
model_rf_base = 'model_rf_base.sav'
pickle.dump(pipeline_rf_base, open(model_rf_base, 'wb'))

# load the model from disk
loaded_model_rf_base = pickle.load(open(model_rf_base, 'rb'))
result_rf_base = loaded_model_rf_base.score(X_test, y_test)
print(result_rf_base)

0.25328043942630457


In [ ]:
# rf_gridsearchcv

# save the model to disk
model_rf_gridsearchcv = 'model_rf_gridsearchcv.sav'
pickle.dump(pipeline_rf_gridsearchcv, open(model_rf_gridsearchcv, 'wb'))

# load the model from disk
loaded_model_rf_gridsearchcv = pickle.load(open(model_rf_gridsearchcv, 'rb'))
result_model_rf_gridsearchcv = loaded_model_rf_gridsearchcv.score(X_test, y_test)
print(result_model_rf_gridsearchcv)

In [15]:
# knn_base

# save the model to disk
model_knn_base = 'model_knn_base.sav'
pickle.dump(pipeline_knn_base, open(model_knn_base, 'wb'))

# load the model from disk
loaded_model_knn_base = pickle.load(open(model_knn_base, 'rb'))
result_model_knn_base = loaded_model_knn_base.score(X_test, y_test)
print(result_model_knn_base)

0.24397314617027768


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.